# Week 10 Lab Lecture

## Processing Data with Python, Part 2

### Topics

* exploring and summarizing data
* data cleaning and manipulation
* merging data
* using time in pandas
* working with WPRDC data

# Some more data manipulation

There are a bunch of standard functions provided by pandas for manipulating data, and now that you've had a chance to get your feet wet, we're going to run through a bunch of things that you probably should know when doing data manipulation with pandas.

## More with exploring and summarizing data

Once your data has been loaded as a Dataframe, you can start using Pandas various functions to quickly explore your data 

In [ ]:
# load pandas
import pandas as pd
import numpy as np

# load data
center_attendance_pandas = pd.read_csv("community-center-attendance.csv", 
                                       index_col="_id") # use the column named _id as the row index

### Helpful functions for exploring DataFrames and Series

Functions for looking at parts of the DataFrame include
* `<dataframe>.head(n)` - look at the first n rows of the dataframe
* `<dataframe>.tail(n)` - look at the last n rows of the dataframe
* `<dataframe>.sample(n)` - randomly select n rows from the dataframe

In [ ]:
# Look at the first 10 rows
center_attendance_pandas.head(10)

In [ ]:
# Look at the last 5 rows
center_attendance_pandas.tail()

In [ ]:
# Grab 5 random rows
center_attendance_pandas.sample(5)

Functions that let us count rows and columns include: 
* `<dataframe>.shape` - return the rows and columns as a python data structure (not a function!)
* `<dataframe>.info()` - Display the datatypes of the index and columns as well as memory usage
* `<dataframe>.describe()` - Compute summary statistics for numerical columns

In [ ]:
# How many rows and columns
center_attendance_pandas.shape

In [ ]:
# Inspect the datatypes
center_attendance_pandas.info()

The output above shows us a lot of implementation details about our DataFrame. Data types, number of rows and columns, and the datatype of the column. It also shows us **memory usage**, which is useful because memory is a limited resource.

From there, we can also start doing some computations on the data.

In [ ]:
# Compute summary statistics on the numerical columns
center_attendance_pandas.describe()

The `describe()` function will automatically compute summary statistics for numerical columns and ignore categorical columns.

### Counting Numerical Data

One of the things I mentioned last week is that pandas doesn't do anything that plain old Python can't. We can use traditional Python functions to get information about our Dataframe.

The `len()` function tells us the length of the sequence.

In [ ]:
# use a standard python function to get the length of the sequence
len(center_attendance_pandas)

So, this tells us our dataset has 18,367 rows. But this is just information about the dataset itself, it doesn't tell us how many people visited community centers.

What if we wanted to know the total attendance: how many people visited all the community centers for all time (in the dataset)?

First, let's answer this using pure Python.

In [ ]:
# Load up the CSV module and center attendance in python data structures
import csv

with open('community-center-attendance.csv') as f:
    center_attendance_python = [row for row in csv.reader(f)]

# look at the first ten rows of the data loaded in python
center_attendance_python[0:10]

In [ ]:
# create a variable to hold the total attendance
total_attendance = 0

# loop over the data that was loaded using pure python
for row in center_attendance_python[1:]: # skip the header row using a list slice
    # add the row count to the total, convert string to int
    row_attendance = int(row[3])
    total_attendance = total_attendance + row_attendance

print(total_attendance)

Now, here is how we do the exact same thing with Pandas.

This code selects the `attendance_count` column and then computes the sum of all the values.

In [ ]:
# compute the total attendance with the pandas sum function
center_attendance_pandas['attendance_count'].sum()

We can also look at the summary statistics individually, column-by-column.
* `<dataframe>[<column name>].mean()` - calculate the mean value for the column values
* `<dataframe>[<column name>].std()` - calculate the standard deviation for the column values
* `<dataframe>[<column name>].var()` - calculate the variance value for the column values
* `<dataframe>[<column name>].median()` - calculate the median value for the column values
* `<dataframe>[<column name>].min()` - calculate the minimum value for the column values

In [ ]:
# mean attendance per day at all community centers
center_attendance_pandas['attendance_count'].mean()

In [ ]:
# standard deviation
center_attendance_pandas['attendance_count'].std()

In [ ]:
# variance
center_attendance_pandas['attendance_count'].var()

In [ ]:
# median attendance per day at all community centers
center_attendance_pandas['attendance_count'].median()

In [ ]:
# minimum attendance at community centers
center_attendance_pandas['attendance_count'].min()

**NOTE**: missing values are automatically skipped unless the entire column is NaN.

In [ ]:
df = pd.DataFrame([[1.4, None], [7.1, -4.5], 
                  [None, None], [0.75, -1.3]],
                 index=['a','b','c','d'],
                 columns=['one','two'])
df

In [ ]:
df.sum()

In [ ]:
df.sum(axis=1)

In [ ]:
df.mean(axis=1)

In [ ]:
df.mean(axis=1, skipna=False)

### Math Operations

You can do mathematical operations that will get applied to every value in the row or column.

In [ ]:
# create a numerical dataframe
df = pd.DataFrame([[1.4, 4.7], [7.1, -4.5], 
                  [3, 7], [0.75, -1.3]],
                 index=['a','b','c','d'],
                 columns=['one','two'])
df

In [ ]:
# multiple two columns against eachother
df['one'] * df['two']

In [ ]:
# divide a column by a number
df['one'] / 5

However, there are, as you know, two major types of data: numerical and categorical. Pandas is not only a tool for working with numerical data. It has lots of functionality for manipulating categorical data, too.

### Counting Categorical Data

Just like before, we can start counting the distribution of values in the column. 

How many entries do we have per community center? (This isn't counting attendance, it's counting the number of *attendance counts* per center).

First, let's do this in pure Python.

In [ ]:
# Create a dictionary to store the counts
center_counter = dict()

# loop over the data
for row in center_attendance_python[1:]:
    center = row[2]
    
    # check to see if the gender is already in the diction
    if center not in center_counter:
        # create a new entry
        center_counter[center] = 1
    else:
        # increment a new entry
        #center_counter[center] += 1
        center_counter[center] = center_counter[center] + 1

# Display the dictionary 
center_counter

The Pandas way, as usualy, is a bit easier.

In [ ]:
# Do the same thing with pandas
center_attendance_pandas['center_name'].value_counts()

In [ ]:
center_attendance_pandas['center_name'].value_counts().sort_values(ascending=True)

There are a bunch of other functions for working with categorical data.

In [ ]:
center_attendance_pandas['center_name'].unique()

In [ ]:
len(center_attendance_pandas['center_name'].unique())

# Data wrangling, databases, and subsetting

It is sometimes helpful to think of a Pandas DataFrame as a little database. There is data and information stored in the Pandas DataFrame (or Series) and you want to *retrieve* it.

Pandas has multiple mechanisms for getting specific bits of data and information from its data structures. 

## Masking: Filtering by Values

The most common is to use *masking* to select just the rows you want. 

Masking is a two stage process, first you create a sequence of boolean values based upon a conditional expression—which you can think of as a "query"—and then you index your dataframe using that boolean sequence. 

In [ ]:
# Let's look at the chipotle order data
order_data.head(10)

In [ ]:
# Let's look at all the columns
order_data.info()

What if we only wanted to look at a **specific order**? Let's try and isolate just the orders for chicken bowls.

Firstly, create a *query mask*, a list of `True/False` values for rows that satisfy a particular condition.

In [ ]:
# create a query mask for chicken bowls
query_mask = order_data['item_name'] == "Chicken Bowl"

#look at the first 20 items to see what matches
query_mask.head(20)

This tells us the *row id* and True or False if the item is a chicken bowl.

Let's just test to see if this is correct:

In [ ]:
order_data.iloc[19]

Yup! So now that we know the mask works, we can create a *subset* of our data containing chicken bowls.

In [ ]:
chicken_bowls = order_data[query_mask]
chicken_bowls.head()

Now, you've got just the orders for chicken bowls, so you can do data operations on *just* those orders.

In [ ]:
# Calculate the mean price for chicken bowls
chicken_bowls['item_price'].mean()

In [ ]:
# See how many chicken bowls people order
chicken_bowls['quantity'].value_counts()

We can also combine query masks using **boolean logic**. Can we look at just the chicken bowl orders that were less than $10?

In [ ]:
# create a query mask for chicken bowls
item_query_mask = order_data['item_name'] == "Chicken Bowl"
# create a query mask for cheap orders
price_query_mask = order_data['item_price'] < 10

# apply both query masks using boolean AND
cheap_chicken_bowls = order_data[item_query_mask & price_query_mask]
cheap_chicken_bowls.head()

In [ ]:
# Median price for cheap chicken bowls
cheap_chicken_bowls['item_price'].median()

Query masks can be used to filter and create subsets of data.

**NOTE**: this method of subsetting data creates what is called a "view" of the data. You are basically working with a big slice of the original dataframe, *not* a separate copy of the data.

This means if you try an do transformations on that view, you will get an error. For more information, [see the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy).

If you do want to do transformations on this data, it's trivial to make a copy, though.

In [ ]:
cheap_chicken_bowls['half_price'] = cheap_chicken_bowls['item_price'] / 2

In [ ]:
copy_of_cheap_chicken_bowls = cheap_chicken_bowls.copy()
copy_of_cheap_chicken_bowls['half_price'] = copy_of_cheap_chicken_bowls['item_price'] / 2
copy_of_cheap_chicken_bowls.head()

# Merging Data

Bringing disparate datasets together is one of the more powerful features of pandas.

Like with Python lists, you can `append()` and `concat()` pandas Series and Dataframes. The `concat` is a module function, you call it directly from the pandas module (usually called `pd`)

In [ ]:
# concatinate two series together
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2]) 
# note the Series are passed as a list

In [ ]:
# order matters
pd.concat([ser2, ser1])

In [ ]:
# concatinate two dataframes
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2])

Pandas will automatically handle lining up matching indices.

In [ ]:
# concatinate dataframes horizontally
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"C":["C1", "C2"],
                    "D":["D1","D2"]},index=[1,2])
pd.concat([df1,df2], axis=1)

And pandas will gracefully handle misalignment.

In [ ]:
# What happens when indexes don't line up
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], axis=1)

In [ ]:
# create a hierarchical index
df1 = pd.DataFrame({"A":["A1", "A2"],
                    "B":["B1","B2"]},index=[1,2])
df2 = pd.DataFrame({"A":["A3", "A4"],
                    "B":["B3","B4"]},index=[3,4])
pd.concat([df1,df2], keys=["df1", 'df2'])

The `append` function is a method of a Series/Dataframe and returns a new object.

In [ ]:
# append df2 to df1
df1.append(df2)

## Joins: more powerful concatenation

While `concat()` is useful, it lacks the power to do complex data merging.

For example, let's say I have two tables of different data but one overlapping column.

This is where the `merge()` function becomes useful because it lets you *join* datasets

The concept of "join" has lots of theory and is a richly developed method for *joining* data.

### One-to-one joins

In [ ]:
# create two dataframes with one shared column
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue', "Nancy"],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR', "Librarian"]})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

In [ ]:
# display df1
df1

In [ ]:
# display df2
df2

In [ ]:
# merge df1 and df2 into a new dataframe df3
df3 = pd.merge(df1, df2)
df3

The new dataframe `df3` now has all of the data from df1 and df2.

The `merge` function automatically connected the two tables on the "employee" column.

But what happens when your data don't line up?

### Many-to-one joins

Sometimes, there isn't a one to one relationshp between rows in the two datasets.

A *many-to-one* join lets you combine these datasets.

In [ ]:
df3

In [ ]:
# make another dataframe about the supervisor for each group
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4

In [ ]:
# Merge df3 from above with the supervisor info in df4
pd.merge(df3,df4)

Notice how the information about Guido, the manager for Engineering, is repeated.

Pandas automatically fills in these values to maintain the tabular, 2 dimensional structure of the data.

While this might seem like duplicated data, it makes it easier to quickly look up Jake and Lisa's supervisor without consulting multiple tables. It's sometimes better to duplicate data to have it arranged in a way that you want.

Now, let's make it even more complicated.

### Many-to-many joins

Let's combine the employee information with skills information.

Notice there isn't a one-to-one or even a one-to-many relationship between these tables.

Each group can have multiple skills, so **what do you think will happen?**

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# create a new dataframe with skills information
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR', 'Librarian'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization', 'nunchucks']})
df5

In [ ]:
# what's going to happen?
pd.merge(df1, df5)

Amazing!

![whoa dude](https://media.giphy.com/media/Lcn0yF1RcLANG/giphy.gif)

Pandas merge capabilities are very useful.

But what do you do if the names of your columns don't match? You could change column names.

But that's crazy! Just use the `left_on` and `right_on` parameters to the `merge()` function.

In [ ]:
# Use the employee table specified above
df1

In [ ]:
# Create a new salary table, but use "name" instead of "employee" for the column index
df3 = df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue','Nancy'],
                    'salary': [70000, 80000, 120000, 90000,1000000]})
df3

In [ ]:
# lets try and merge them without specifying what to merge on
pd.merge(df1, df3)

What are the column names I should specify here?

In [ ]:
# Now lets specify the column name 
pd.merge(df1, df3, left_on="employee", right_on="name" )

Notice we now have a redundant employee/name column; this is a by-product of merging different columns.

If you want to get rid of it, that's trivial: you can use the `drop` method.

In [ ]:
# drop the name column, axis=1 means axis='col', which is confusing
pd.merge(df1, df3, left_on="employee", right_on="name" ).drop("name", axis=1)

Like I said, there is a ton of theory around merging and joining data, so this is just us dipping our toes in.

# `groupby()` and aggregation

We looked at the `groupby()` function last week, and I don't want to waste time on it today. Essentially, `groupby()` lets you group all of the items in a DataFrame by a single column.

There's a bunch of other ways to sort data in DataFrames with pandas. The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``size()``               | Total number of items w/ NaNs   |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all functions of ``DataFrame`` and ``Series`` objects.

You can also do multiple levels of grouping with something called [Multilevel Indexing](https://pandas.pydata.org/pandas-docs/stable/advanced.html). Unfortunately, we don't have time to go in depth into this, but the Python Data Science Handbook (one of your textbooks for the course) has a great intro to the topic in the chapter [Hierarchical Indexing](https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html).


# Now, let's work with some real data

The WPRDC, which stands for Western Pennsylvania Regional Data Center, is *the* place to go for data around Pittsburgh. You'll be working with WPRDC data for your final project (*wink, wink*).

So, let's grab the [Allegheny County Jail's daily census](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census) from the WPRDC.

In [ ]:
# Grab three months of data
january17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/3b5d9c45-b5f4-4e05-9cf1-127642ad1d17")
feburary17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/cb8dc876-6285-43a8-9db3-90b84eedb46f")
march17_jail_census = pd.read_csv("https://data.wprdc.org/datastore/dump/68645668-3f89-4831-b1de-de1e77e52dd3")


In [ ]:
january17_jail_census.head()

In [ ]:
# Use the concat function to combine all three into one dataframe
# Remember I need to make a list of the all the dataframes for
# the concat fuction
jail_census = pd.concat([january17_jail_census, 
                         feburary17_jail_census, 
                         march17_jail_census])
jail_census

Even though these aren't *really* valid computations because we are looking at a daily census, we can still use these data for demonstration purposes.

In [ ]:
# Compute the average age at booking by gender
jail_census.groupby('Gender')['Age at Booking'].mean()

In [ ]:
# compute the average age at booking by race then gender 
jail_census.groupby(['Race', 'Gender'])['Age at Booking'].mean()

If we look at the [data dictionary](https://data.wprdc.org/dataset/allegheny-county-jail-daily-census/resource/f0550174-16b0-4f6e-88dc-fa917e74b56c), we can see the following mapping for race categories:
```
Race of Inmate
A-ASIAN OR PACIFIC ISLANDER
B-BLACK OR AFRICAN AMERICAN
H-HISPANIC 
I-AMERICAN INDIAN OR ALASKAN NATIVE
U-UNKNOWN
W-WHITE
```
The `x` category hasn't been described.

In [ ]:
# how many total rows in the dataset have "x" for race
jail_census['Race'].value_counts()['x']

In [ ]:
# Get the statistical summary of age at booking by gender
jail_census.groupby('Gender')['Age at Booking'].describe()

In [ ]:
# Compute the difference between Age at Booking and current age
age_difference = jail_census['Current Age'] - jail_census['Age at Booking']
age_difference.value_counts()

Let's sort them by date, so we can see who was there on any given day.

In [ ]:
jail_census.groupby('Date').count()

In [ ]:
jail_census['year'] = jail_census['Date'].str.split("-").str[0]
jail_census['month'] = jail_census['Date'].str.split("-").str[1]
jail_census['day'] = jail_census['Date'].str.split("-").str[2]

jail_census.head()

In [ ]:
jail_census.groupby('month').count()

In [ ]:
jail_census.groupby('day').count()

This is a really awkward way of dealing with time. We shouldn't have to make a separate column for year, month, day.

There must be a better way to do this time stuff... any ideas?

# One more thing

Before we get into doing some real-world data manipulation, I want to briefly dive into working with time in pandas. One of the most powerful features of pandas is its time series functionality.

Dates and time are a Python and pandas data type (like integers and strings). By using the `datetime` data types you can do advanced, time-centric analysis.

One thing to remember about computers is they are *very* specific. The following things are *all different* to the computer:

* **time stamps** - a specific moment in time (July 4th, 2017 at 7:52am and 34 seconds)
* **time intervals** - a length of time with start and end points (the year 2017)
* **time duration** - a specific length of time (a year, a month, a day)
   
Pandas has its own data types for time (much like Series and DataFrame). If you have a lot of dates, it is often useful to use the Pandas functions over the native Python functions. Pandas is most powerful when you index by time using the `DatetimeIndex`.

In [ ]:
# Create a Series with a DateTime index
index = pd.DatetimeIndex(['2014-03-04', '2014-08-04',
                          '2015-04-04', '2015-09-04',
                          '2016-01-01', '2016-02-16'])
data = pd.Series([0, 1, 2, 3, 4, 5], index=index)
data

Now that the index is made of DateTimes, we can index using date strings.

**NOTE**: this only works on strings.

In [ ]:
# grab the value for a specific day
data["2015-04-04"]

In [ ]:
# grab a slice between two dates
data['2014-08-01':'2016-01']

In [ ]:
# give me everything from 2015
data['2015']

Pandas has some functions to make parsing dates easy, as well.

In [ ]:
# use the to_datetime function instead of the parser function
date = pd.to_datetime("4th of July, 2017")
date

In [ ]:
# use string format codes to get the weekday
date.strftime("%A")

In [ ]:
# give me today's date
today = pd.to_datetime("today")
today

That is the day, but also the exact time. Remember? Computers are picky. Timestamps must always be a *specific moment*.

## Working with Time on Real Data

Let's look at the [311 data for the city of Pittsburgh](https://data.wprdc.org/dataset/311-data) from the WPRDC. Did you know, you can give the URL directly to Pandas!

In [ ]:
# load the 311 data directly from the WPRDC
pgh_311_data = pd.read_csv("https://data.wprdc.org/datastore/dump/76fda9d0-69be-4dd5-8108-0de7907fc5a4")
pgh_311_data.head()

In [ ]:
# Inspect the dataframe and Pandas automatic data type detection
pgh_311_data.info()

Okay, now we have the data, but we need it to be indexed by date.

* **What column has the date information?**
* **What format do you think that column is currently in?**
* **What function might we use to convert that column into dates?**

In [ ]:
pgh_311_data['CREATED_ON'].head()

In [ ]:
# convert the "CREATED_ON" column to dates
pd.to_datetime(pgh_311_data['CREATED_ON']).head()

We can convert the "CREATED_ON" column to Pandas `datetime` objects, and now we have to set that to the dataframe's index.

In [ ]:
# set the index of pgh_311_data to be the parsed dates in the "CREATED_ON" column
pgh_311_data.index = pd.to_datetime(pgh_311_data['CREATED_ON'])
pgh_311_data.head()

Huh, now we have CREATED_ON twice. That isn't very tidy. We can also skip this extra conversion step entirely by specifying the index column and date parsing in `read_csv()` function call.

In [ ]:
# load the 311 data directly from the WPRDC and parse dates directly
pgh_311_data = pd.read_csv("https://data.wprdc.org/datastore/dump/76fda9d0-69be-4dd5-8108-0de7907fc5a4",
                           index_col="CREATED_ON", 
                           parse_dates=True)
pgh_311_data.head()

In [ ]:
pgh_311_data.info()

Now that the dataframe has been indexed by time, we can select 311 complains by time!

In [ ]:
# Select 311 complaints on January 1st, 2016
pgh_311_data['2016-01-01']

In [ ]:
# Select the times just around the new years celebration
pgh_311_data["2015-12-31 20:00:00":"2016-01-01 02:00:00"]

Someone had a fun New Year's. 

## Resampling

Last week, we "smoothed over" the community center data by using the `resample()` method. Let's do that some more, shall we?

In [ ]:
# count the number of complaints per month
pgh_311_data.resample("W").size()

In [ ]:
# compute the mean of complaints per quarter...note this doesn't make sense, but works anyway
pgh_311_data.resample("Q").mean()

Let's plot this. 

In [ ]:
# Tell matplotlib to render plots inline
%matplotlib inline

In [ ]:
# Create a graph of the monthly complaint counts
pgh_311_data['REQUEST_ID'].resample("M").count().plot();

Try the code above, but re-sampling based upon different date periods. The strings for specifying an offset are located [here](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases) and below:

|Alias|Description|
|-----|-----------|
|B|business day frequency|
|C|custom business day frequency|
|D|calendar day frequency|
|W|weekly frequency|
|M|month end frequency|
|SM|semi-month end frequency (15th and end of month)|
|BM|business month end frequency|
|CBM|custom business month end frequency|
|MS|month start frequency|
|SMS|semi-month start frequency (1st and 15th)|
|BMS|business month start frequency|
|CBMS|custom business month start frequency|
|Q|quarter end frequency|
|BQ|business quarter end frequency|
|QS|quarter start frequency|
|BQS|business quarter start frequency|
|A, Y|year end frequency|
|BA, BY|business year end frequency|
|AS, YS|year start frequency|
|BAS, BYS|business year start frequency|
|BH|business hour frequency|
|H|hourly frequency|
|T, min|minutely frequency|
|S|secondly frequency|
|L, ms|milliseconds|
|U, us|microseconds|
|N|nanoseconds|

In [ ]:
# Try a different resampling here

pgh_311_data['REQUEST_ID'].resample("SOMETHING DIFFERENT").count().plot();

In [ ]:
# Try yet another resampling here

pgh_311_data['REQUEST_ID'].resample("EVEN MORE DIFFERENT").count().plot();

# Wrapping up

Okay, so that's pandas. We'll get a little bit more into some specifics next week when we talk about data visualization. We'll be using pandas, geopandas, and matplotlib to answer some interesting questions with data. 